In [ ]:
!cat /etc/issue.net # OS
!cat /proc/cpuinfo  # CPU
!cat /proc/meminfo  # RAM
!df -h              # Disk
!nvidia-smi         # GPU

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

!cd "gdrive/My Drive/Colab Notebooks"; ls;

In [ ]:
!cp "gdrive/My Drive/Colab Notebooks/Intro2AI-HW/requirements.txt" .
!pip install -r requirements.txt

In [ ]:
import os
import sys
REPOSITORY_PATH = os.path.join('content', 'gdrive', 'My Drive', 'Colab Notebooks', 'Intro2AI-HW')
sys.path.append(REPOSITORY_PATH)

!cp -r "gdrive/My Drive/Colab Notebooks/Intro2AI-HW/PyGame-Learning-Environment" .
!cd "PyGame-Learning-Environment"; pip install -e .;
sys.path.append('PyGame-Learning-Environment')
!cp -r "gdrive/My Drive/Colab Notebooks/Intro2AI-HW/utils" .
from utils.env import Environment
!cp -r "gdrive/My Drive/Colab Notebooks/Intro2AI-HW/agent" .
from agent.agent import Agent

In [ ]:
import time

os.environ["SDL_VIDEODRIVER"] = "dummy"

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
if torch.__version__ <= '1.1.0':
    from tensorboardX import SummaryWriter
else:
    from torch.utils.tensorboard import SummaryWriter
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from google.colab import output

In [ ]:
COLAB_NOTEBOOKS_PATH = os.path.join('.', 'gdrive', 'My Drive', 'Colab Notebooks', 'Intro2AI-HW')
CHECKPOINT_PATH = os.path.join(COLAB_NOTEBOOKS_PATH, 'checkpoint.pth.tar')
BEST_CHECKPOINT_PATH = os.path.join(COLAB_NOTEBOOKS_PATH, 'best_checkpoint.pth.tar')
SAVED_CHECKPOINT_PATH = 'checkpoint.pth.tar'
SAVED_BEST_CHECKPOINT_PATH = 'best_checkpoint.pth.tar'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device:', device)


def train(hParam, env, agent):
    best = 0
    global_steps = 0
    i_episode = 0

    writer = SummaryWriter()
    print('TRAIN STARTS')

    while(hParam['MAX_ITER'] > global_steps ):
        # Initialize the environment and state
        env.reset()
        state = env.start()
        i_episode += 1

        while not env.game_over():
            global_steps += 1

            action = agent.getAction(state)
            next_state, reward, done = env.step(action)

            # PyGameDisplay to OpenCV
            frame = env.get_screen()
            frame = np.rot90(frame, k=1)
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            frame = frame[::-1]

            output.clear()
            cv2_imshow(frame)
            # time.sleep(0.1)
            # output.clear()

            # Store the state, action, next_state, reward, done in memory
            agent.memory.push(state, action, next_state, reward, done)
            
            if global_steps > hParam['BUFFER_SIZE']:
                if global_steps % hParam['TARGET_UPDATE'] == 0:
                    agent.updateTargetNet()

                # Update the target network, copying all weights and biases in DQN
                if env.game_over():
                    print('Episode: {}  Global Step: {}, Episode score: {:.4f}  Episode Total Reward: {:.4f} Loss: {:.4f}'.format(
                       i_episode, global_steps, env.getScore() ,env.total_reward, loss))

                    writer.add_scalar('Episode_total_reward', env.total_reward, i_episode)
                    writer.add_scalar('Episode', env.getScore(), i_episode)

                    agent.save(CHECKPOINT_PATH)

                    if env.total_reward > best:
                        agent.save(BEST_CHECKPOINT_PATH)
                        best = env.total_reward            
                
                loss = agent.updateQnet()
                writer.add_scalar('train_loss', loss, global_steps)
            
            elif global_steps%500 == 0: 
                print('steps {}/{}'.format(global_steps, hParam['MAX_ITER']))
                
            # Move to the next state
            state = next_state


In [ ]:
from agent.agent import Agent

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# if __name__ == "__main__":
hParam = {
    'BATCH_SIZE': 32,
    'GAMMA': 0.99,
    'TARGET_UPDATE': 10000,
    'EPS_START': 0.1,
    'EPS_END': 0.0001,
    'MAX_ITER': 2000000,
    'DISCOUNT_FACTOR': 0.99,
    'LR': 1e-6,
    'MOMENTUM': 0.9,
    'BUFFER_SIZE': 30000
}

env = Environment(device, display=True)
sungjun = Agent(env.action_set, hParam)

!cp -r "gdrive/My Drive/Colab Notebooks/Intro2AI-HW/checkpoint.pth.tar" .
!cp -r "gdrive/My Drive/Colab Notebooks/Intro2AI-HW/best_checkpoint.pth.tar" .

if os.path.exists(SAVED_CHECKPOINT_PATH):
    try:
        sungjun.load(SAVED_CHECKPOINT_PATH)
    except:     # RuntimeError
        if os.path.exists(SAVED_BEST_CHECKPOINT_PATH):
            sungjun.load(SAVED_BEST_CHECKPOINT_PATH)

train(hParam, env, sungjun)